In [1]:
from data_reader import DataReader
from ideology_and_objective import IdeologicalObjectiveModule
from get_longevity import LongevityModule
from get_domestic_support import DSupModule
import pickle
import pandas as pd

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
data_obj = DataReader()

c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3337: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [4]:
df = data_obj.get_pandas_df()
longevity_obj = LongevityModule()

c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3417: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
ds_obj = DSupModule()

In [6]:
df.head(2)

,Triadid,DyadID,ConflictID,Startdate1,Startdate2,Target,TarNum_COW,NAG_name,NAGcode_1,NAGcode_2,...,DS_WeaponLog,DS_Precision_WL,DS_FinAid,DS_Precision_FinAid,DS_Transport,DS_Precision_Tp,DS_Other,DS_Precision_Oth,statesup,defacto
0,775026800,322.0,24.0,20.03.1990,20.03.1990,Myanmar,775,ABSDF,26,1026.0,...,0.0,4.0,0.0,4.0,0.0,4.0,0.0,4.0,1,0
1,775026800,322.0,24.0,20.03.1990,20.03.1990,Myanmar,775,ABSDF,26,1026.0,...,0.0,4.0,0.0,4.0,0.0,4.0,0.0,4.0,1,0


In [7]:
with open("pickles/long_horizon_list.pickle", "rb") as f:
    long_horizon_list = pickle.load(f)
    
with open("pickles/short_horizon_list.pickle", "rb") as f:
    short_horizon_list = pickle.load(f)

In [24]:
new_df = pd.DataFrame(columns=["nagid", "long_horizon", "domsup", "ppd", "lifespan"])

In [25]:
new_df['nagid'] = df['NAGcode_1'].unique().tolist()

In [26]:
nagids = new_df['nagid'].tolist()

In [27]:
long_horizon_or_not = []
for nagid in nagids:
    temp = 0
    if nagid in set(long_horizon_list):
        temp = 1
    long_horizon_or_not.append(temp)

new_df['long_horizon'] = long_horizon_or_not

In [28]:
domsup_or_not = []
for nagid in nagids:
    domsup = ds_obj.get(nagid)
    domsup_or_not.append(domsup[0])

new_df['domsup'] = domsup_or_not

In [29]:
ppd_or_not = []
for nagid in nagids:
    ppd = ds_obj.get_political_party(nagid)
    ppd_or_not.append(ppd[0])

new_df['ppd'] = ppd_or_not

In [30]:
lifespan = []
for nagid in nagids:
    l = longevity_obj.calculate_longevity(nagid)
    lifespan.append(l[0])

new_df['lifespan'] = lifespan

In [31]:
new_df.to_csv("files/nagid_domsup_ppd_horizon_lifespan.csv", index=False)

In [32]:
new_df = pd.read_csv("files/nagid_domsup_ppd_horizon_lifespan.csv")

In [33]:
new_df

,nagid,long_horizon,domsup,ppd,lifespan
0,26,0,0,0,32
1,202,1,0,0,44
2,301,0,0,0,24
3,235,0,1,0,2
4,345,0,0,0,4
...,...,...,...,...,...
531,553,0,0,0,5
532,500,0,0,0,6
533,566,0,0,0,14
534,530,1,1,0,5


In [40]:
def domsup_ppd(df, nagids):
    return_dict = {}
    df = df[df['nagid'].isin(nagids)]
    f1 = (df['domsup']==1) 
    f2 = (df['ppd'] == 1)
    df1 = df[f1]
    df2 = df[f2]
    df3 = df[f1 & f2]
    return_dict = {
        "domsup_nags": (df1.shape[0],df1['lifespan'].mean(),df1['lifespan'].std()),
        "ppd_nags": (df2.shape[0],df2['lifespan'].mean(),df2['lifespan'].std()),
        "both_nags":(df3.shape[0],df3['lifespan'].mean(),df3['lifespan'].std())
    }
    return return_dict

def neither_domsup_ppd(df, nagids):
    return_dict = {}
    df = df[df['nagid'].isin(nagids)]
    f1 = (df['domsup']==0) 
    f2 = (df['ppd'] == 0)
    df1 = df[f1]
    df2 = df[f2]
    df3 = df[f1 & f2]
    return_dict = {
        "no_domsup_nags": (df1.shape[0],df1['lifespan'].mean(),df1['lifespan'].std()),
        "no_ppd_nags": (df2.shape[0],df2['lifespan'].mean(),df2['lifespan'].std()),
        "neither_nags":(df3.shape[0],df3['lifespan'].mean(),df3['lifespan'].std())
    }
    return return_dict

def info(df, nagids):
    t = domsup_ppd(df, nagids)
    t2 = neither_domsup_ppd(df, nagids)
    return t, t2

In [41]:
info(new_df, short_horizon_list)

({'domsup_nags': (50, 13.24, 12.344642727986608),
  'ppd_nags': (42, 17.5, 13.620733836467933),
  'both_nags': (12, 20.083333333333332, 15.281946171475637)},
 {'no_domsup_nags': (278, 9.856115107913668, 13.019042510323843),
  'no_ppd_nags': (286, 9.325174825174825, 12.544483479155392),
  'neither_nags': (248, 9.056451612903226, 12.813708764336903)})

In [42]:
info(new_df, long_horizon_list)

({'domsup_nags': (57, 18.385964912280702, 15.226567930013589),
  'ppd_nags': (34, 19.852941176470587, 16.992212749305107),
  'both_nags': (15, 23.866666666666667, 18.360931608276566)},
 {'no_domsup_nags': (151, 14.748344370860927, 14.10872946939653),
  'no_ppd_nags': (174, 14.942528735632184, 13.847940788751407),
  'neither_nags': (132, 14.469696969696969, 13.92479220941593)})

In [ ]:
new_df[new_df['domsup']==1].shape

In [ ]:
nagwithdomsup = new_df[new_df['domsup']==1]['nagid'].tolist()

In [ ]:
domsup_nags = [longevity_obj.calculate_longevity(i)[0] for i in nagwithdomsup]
get_mean_and_std(domsup_nags)

In [ ]:
new_df[new_df['domsup']==0].shape

In [ ]:
nagwithoutdomsup = new_df[new_df['domsup']==0]['nagid'].tolist()
withoutdomsup_nags = [longevity_obj.calculate_longevity(i)[0] for i in nagwithoutdomsup]
get_mean_and_std(withoutdomsup_nags)

In [ ]:
# without dom and short

new_df[(new_df['domsup']==0)&(new_df['long_horizon']==0)].shape

In [ ]:
nagwithoutdomsup_short = new_df[(new_df['domsup']==0)&(new_df['long_horizon']==0)]['nagid'].tolist()
withoutdomsup_nags_short = [longevity_obj.calculate_longevity(i)[0] for i in nagwithoutdomsup_short]
get_mean_and_std(withoutdomsup_nags_short)

In [ ]:
new_df[(new_df['domsup']==0)&(new_df['long_horizon']==1)].shape

In [ ]:
nagwithoutdomsup_longg = new_df[(new_df['domsup']==0)&(new_df['long_horizon']==1)]['nagid'].tolist()
withoutdomsup_nags_longg = [longevity_obj.calculate_longevity(i)[0] for i in nagwithoutdomsup_longg]
get_mean_and_std(withoutdomsup_nags_longg)

In [ ]:
# domsup and short
new_df[(new_df['domsup']==1)&(new_df['long_horizon']==0)].shape

In [ ]:
nagwithdomsup_short = new_df[(new_df['domsup']==1)&(new_df['long_horizon']==0)]['nagid'].tolist()
withdomsup_nags_short = [longevity_obj.calculate_longevity(i)[0] for i in nagwithdomsup_short]
get_mean_and_std(withdomsup_nags_short)

In [ ]:
# domsup and long

new_df[(new_df['domsup']==1)&(new_df['long_horizon']==1)].shape

In [ ]:
nagwithdomsup_long = new_df[(new_df['domsup']==1)&(new_df['long_horizon']==1)]['nagid'].tolist()
withdomsup_nags_long = [longevity_obj.calculate_longevity(i)[0] for i in nagwithdomsup_long]
get_mean_and_std(withdomsup_nags_long)

In [ ]:
group_1 = {'demands_for_autonomy', 'territorial_demand'}
short_horizon_list, long_horizon_list = [], []
for nag_id in df['NAGcode_1'].unique():
    objs = obj.get_objective(nag_id)
    long_horizon = 1 if set(objs).intersection(group_1) != set() else 0
    if long_horizon == 1:
        long_horizon_list.append(nag_id)
    elif long_horizon == 0:
        short_horizon_list.append(nag_id)
    else:
        print(nag_id, objs, set(objs).intersection(group_1), long_horizon)

In [ ]:
len(long_horizon_list), len(short_horizon_list)

In [ ]:
long_nags = [longevity_obj.calculate_longevity(i)[0] for i in long_horizon_list]
short_nags =[longevity_obj.calculate_longevity(i)[0] for i in short_horizon_list]

all_nags = [longevity_obj.calculate_longevity(i)[0] for i in df['NAGcode_1'].unique()]

In [ ]:
def get_mean_and_std(l):
    return np.mean(l), np.std(l)

In [ ]:
get_mean_and_std(long_nags)

In [ ]:
get_mean_and_std(short_nags)

In [ ]:
get_mean_and_std(all_nags)

In [ ]:
len(long_horizon_list)

In [ ]:
len(all_nags)

In [ ]:
from ideology_and_objective import OBJECTIVE_MAP


In [ ]:
OBJECTIVE_MAP

In [ ]:
with open("pickles/long_horizon_list.pickle", "wb") as f:
    pickle.dump(long_horizon_list, f)

In [ ]:
with open("pickles/short_horizon_list.pickle", "wb") as f:
    pickle.dump(short_horizon_list, f)

In [ ]:
df.columns

In [ ]:
df['Tar_DomSup']